In [ ]:
   """https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ModuleNotFoundError: ignored

In [ ]:
res # score == F1 Score: it's computed on the individual words in the p

In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline


# Load the PDF file and extract the text from it
from pypdf import PdfReader
reader = PdfReader('stock_market2.pdf')
pages_text = []
for page_num, page in enumerate(reader.pages):
    parts = []
    def visitor_body(text, cm, tm, font_dict, font_size):
        parts.append(text)
    page.extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
    # remove page numbers
    text_body = re.sub(r'Page \d+ of \d+', '', text_body)
    # remove first line (if any) on subsequent pages
    if page_num > 0:
        text_body = text_body.split('\n', 1)[-1]
    page_text = f"Page {page_num+1}:\n{text_body}\n"
    pages_text.append(page_text)
text = '\n'.join(pages_text)
# Join the text from all pages to create a single string
context = " ".join(pages_text)


# Fine-tune the model on the extracted text for question answering
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
nlp.context = text

# Example question
question = "What is SENSEX?"

# Get the answer to the question
result = nlp(question=question, context=context)

# Print the answer
print(result)


In [ ]:
pages_text

In [ ]:
# question = "What is the stock market?"
# text = "The stock market is a collection of markets where stocks are traded, among other financial products."
# answer = extract_answer(question, text)
# print(answer)


In [ ]:
from pypdf import PdfReader
import re
from transformers import pipeline



reader = PdfReader('stock_market2.pdf')

pages_text = []

for page_num, page in enumerate(reader.pages):
    parts = []
    
    def visitor_body(text, cm, tm, font_dict, font_size):
        parts.append(text)

    page.extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
    # remove page numbers
    text_body = re.sub(r'Page \d+ of \d+', '', text_body)
    # remove first line (if any) on subsequent pages
    if page_num > 0:
        text_body = text_body.split('\n', 1)[-1]

    page_text = f"Page {page_num+1}:\n{text_body}\n"
    print(page_text)
    pages_text.append(page_text)



# qa_model = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')
# document = pages_text
# question = 'What are future contracts?'
# answer = qa_model({'context': document, 'question': question})

# print(answer['answer'])

In [ ]:
pages_text

In [ ]:
# ! pip install torch
# ! pip install transformers
# ! pip install pypdf
# ! pip install svgwrite
# ! pip install transformers
!pip install allennlp==2.8.0
!pip install transformers==4.6.1


In [ ]:
! git clone https://github.com/allenai/allennlp-bert-qa-wrapper.git

In [ ]:
# ! pip install -r requirements.txt
!pip uninstall allennlp
!pip install allennlp


In [ ]:
import json
import torch
from allennlp.data.tokenizers import WordTokenizer
from allennlp.data.fields import TextField
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.data.tokenizers.word_splitter import BertBasicWordSplitter
from allennlp.modules.token_embedders import PretrainedBertEmbedder
from allennlp.models import Model
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BasicIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import Predictor
from allennlp.nn.util import get_text_field_mask

from pypdf import PdfReader

# Define dataset reader
class QADatasetReader(DatasetReader):
    def __init__(self, tokenizer, token_indexers, lazy=False):
        super().__init__(lazy)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers

    def text_to_instance(self, question, answer):
        question_tokens = self.tokenizer.tokenize(question)
        answer_tokens = self.tokenizer.tokenize(answer)
        fields = {
            "question": TextField(question_tokens, self.token_indexers),
            "answer": TextField(answer_tokens, self.token_indexers),
        }
        return Instance(fields)

    def _read(self, file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
            for item in data:
                question = item['question']
                answer = item['answer']
                yield self.text_to_instance(question, answer)

# Define model
class QAModel(Model):
    def __init__(self, embedder, encoder, dropout):
        super().__init__(vocab=None)
        self.embedder = embedder
        self.encoder = encoder
        self.dropout = torch.nn.Dropout(p=dropout)
        self.linear_layer = torch.nn.Linear(in_features=self.encoder.get_output_dim(),
                                             out_features=2)

    def forward(self, question, answer):
        mask_question = get_text_field_mask(question)
        mask_answer = get_text_field_mask(answer)

        embedded_question = self.embedder(question)
        embedded_answer = self.embedder(answer)

        encoded_question = self.encoder(embedded_question, mask_question)
        encoded_answer = self.encoder(embedded_answer, mask_answer)

        concatenated = torch.cat((encoded_question, encoded_answer), dim=1)
        dropped = self.dropout(concatenated)

        logits = self.linear_layer(dropped)
        probs = torch.nn.functional.softmax(logits, dim=-1)

        return {"probs": probs}

# Fine-tune model
tokenizer = WordTokenizer(BertBasicWordSplitter())
token_indexer = PretrainedBertIndexer(pretrained_model="bert-base-uncased")
dataset_reader = QADatasetReader(tokenizer=tokenizer, token_indexers={"tokens": token_indexer})
train_data = dataset_reader.read("path/to/train/data.json")
dev_data = dataset_reader.read("path/to/dev/data.json")

bert_embedder = PretrainedBertEmbedder(pretrained_model="bert-base-uncased")
bert_encoder = BertEncoder(input_dim=bert_embedder.get_output_dim())
model = QAModel(embedder=bert_embedder, encoder=bert_encoder, dropout=0.1)

iterator = BasicIterator(batch_size=32)
iterator.index_with(vocab)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_data,
    validation_dataset=dev_data,
    patience=5,
    num_epochs=20
)
trainer.train()

# Use model for prediction
class QAPredictor(Predictor):
    def _json_to_instance(self, json_dict):
        question = json_dict["question"]
        answer = json_dict["answer"]
        return self._dataset_reader.text_to_instance(question, answer)

predictor = QAPredictor(model=model, dataset_reader=dataset_reader)


reader = PdfReader('stock_market2.pdf')

all_text =""
for page_num, page in enumerate(reader.pages):
    parts = []
    
    def visitor_body(text, cm, tm, font_dict, font_size):
        y = tm[5]
        if y > 50 and y < 720:
            parts.append(text)

    page.extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
    all_text += text_body

    print(f"Page {page_num+1}:\n{text_body}\n")


# Use model to extract answers from PDF
for item in data:
    question = item["question"]
    prediction = predictor.predict_json({"question": question, "answer": all_text})
    predicted_answer = prediction["probs"].argmax()
    print(f"Question: {question}\nAnswer: {predicted_answer}\n")



In [ ]:
import json
import torch
from allennlp.data.tokenizers import PretrainedTransformerTokenizer
from allennlp.data.fields import TextField
from allennlp.data import Instance

# Load pre-trained tokenizer
tokenizer = PretrainedTransformerTokenizer(model_name="bert-base-cased")

# Tokenize text
text = "Some text to be tokenized."
tokens = tokenizer.tokenize(text)

# Convert tokens to fields
text_field = TextField(tokens, token_indexers={"tokens": tokenizer})
fields = {"text": text_field}

# Create instance
instance = Instance(fields)

# Convert instance to tensors
vocab = Vocabulary()
text_field.index(vocab)
tensors = instance.as_tensor_dict()


In [ ]:
from googletrans import Translator

# Read file contents
with open('/content/bonalufestival.txt', 'r') as f:
    text = f.read()

# Translate text to French
translator = Translator()
translated_text = translator.translate(text, dest='fr').text

# Write translated text to new file
with open('translated_file.txt', 'w') as f:
    f.write(translated_text)


TypeError: ignored

In [ ]:
# !pip install googletrans
# !pip install --upgrade googletrans==4.0.0-rc1
!pip install google-cloud-translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import translate_v2 as translate

# Initialize the Translate client
translate_client = translate.Client()

# Translate the word
result = translate_client.translate("నాని", target_language="fr", source_language="te")

# Retrieve the translated text
translated_word = result["translatedText"]

# Print the translated word
print(translated_word)


RefreshError: ignored

In [ ]:
# First, install Fairseq
!pip install fairseq

# Then, download the pre-trained model
!wget https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.ensemble.tar.gz
!tar -xzvf wmt19.en-de.joined-dict.ensemble.tar.gz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-02 13:14:13--  https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.ensemble.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.156.60.69, 108.156.60.51, 108.156.60.36, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.156.60.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11946275315 (11G) [application/gzip]
Saving to: ‘wmt19.en-de.joined-dict.ensemble.tar.gz.1’

wmt19.en-de.joined- 100%[===================>]  11.12G   185MB/s    in 60s     

2023-06-02 13:15:13 (190 MB/s) - ‘wmt19.en-de.joined-dict.ensemble.tar.gz.1’ saved [11946275315/11946275315]

wmt19.en-de.joined-dict.ensemble/
wmt19.en-de.joined-dict.ensemble/model4.pt
wmt19.en-de.joined-dict.ensemble/model3.pt
wmt19.en-de.joined-dict.ensemble/model2.pt
wmt19.en-de.joined-dict.ensemble/model1.pt
wmt19.en-de.joined-dict.ensemble/bpecode

In [ ]:
# First, install Fairseq
!pip install fairseq

# Then, download the pre-trained model
!wget httpsdl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.ensemble.tar.gz
!tar -xzvf wmt19.en-de.joined-dict.ensemble.tar.gz

# Next, load the
from fairseq.models.transformer import TransformerModel
en_de = TransformerModel.from_pretrained(
  'wmt19.en-de.joined-dict.ensemble',
  checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
  data_name_or_path='wmt19.en-de.joined-dict'
)

# Finally, translate a sentence
en_de.translate('Hello, how are you?')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-02 13:34:37--  http://httpsdl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.ensemble.tar.gz
Resolving httpsdl.fbaipublicfiles.com (httpsdl.fbaipublicfiles.com)... failed: Name or service not known.
wget: unable to resolve host address ‘httpsdl.fbaipublicfiles.com’
wmt19.en-de.joined-dict.ensemble/
wmt19.en-de.joined-dict.ensemble/model4.pt
wmt19.en-de.joined-dict.ensemble/model3.pt
wmt19.en-de.joined-dict.ensemble/model2.pt
wmt19.en-de.joined-dict.ensemble/model1.pt
wmt19.en-de.joined-dict.ensemble/bpecodes
wmt19.en-de.joined-dict.ensemble/dict.de.txt
wmt19.en-de.joined-dict.ensemble/dict.en.txt


TypeError: ignored

In [ ]:
from textblob import TextBlob

blob = TextBlob(
    "It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife!"
)
print(blob.translate(to="fr"))

AttributeError: ignored

In [8]:
# !pip install tensor2tensor
# !pip install marian
# !pip install marian-client
# !pip install --upgrade transformers
# !pip uninstall transformers
!pip install transformers
!pip install sentencepiece
!pip install --upgrade googletrans==4.0.0-rc1

# !pip install --upgrade googletrans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=817a98d8db11f6517cdb4148a5291ff0529d2e8f5c94b79b360ff91e9e58c336
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [1]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text(text, src_lang="en", tgt_lang="hi"):
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Tokenize the input text
    input_tokens = tokenizer(text, return_tensors="pt")

    # Generate the translation
    translated_tokens = model.generate(**input_tokens)

    # Decode the translated tokens
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

if __name__ == "__main__":
    # input_text = "Hello,my name is nani is your name also nani and  how are you?"
    input_text = "Hello, how are you?"
    translated_text = translate_text(input_text)
    print(f"Original text: {input_text}")
    print(f"Translated text: {translated_text}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original text: Hello, how are you?
Translated text: हैलो, तुम कैसे हो?


In [2]:
if __name__ == "__main__":
    input_text = "Bonalu is a significant Hindu festival that is celebrated in the southern state of Telangana, India. The festival is dedicated to the worship of the mother goddess, Mahakali, who is believed to protect the people from evil forces. Bonalu is celebrated in the month of Ashada, which falls between July and August. The festival is a symbol of cultural heritage and religious harmony and is celebrated with great enthusiasm by people from all walks of life."
    translated_text = translate_text(input_text)
    print(f"Original text: {input_text}")
    print(f"Translated text: {translated_text}")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original text: Bonalu is a significant Hindu festival that is celebrated in the southern state of Telangana, India. The festival is dedicated to the worship of the mother goddess, Mahakali, who is believed to protect the people from evil forces. Bonalu is celebrated in the month of Ashada, which falls between July and August. The festival is a symbol of cultural heritage and religious harmony and is celebrated with great enthusiasm by people from all walks of life.
Translated text: बाल देवता की उपासना करने का पर्व, भारत के तेलगाना राज्य के दक्षिणी राज्य में मनाया जाता है ।


In [3]:
from transformers import MarianMTModel, MarianTokenizer

def translate_file(file_path, src_lang="en", tgt_lang="hi"):
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Read file contents
    with open(file_path, 'r') as f:
        text = f.read()

    # Split the text into smaller segments
    max_length = model.config.max_position_embeddings - 2  # Account for special tokens
    segments = [text[i:i + max_length] for i in range(0, len(text), max_length)]

    # Initialize translated text
    translated_text = ""

    # Translate each segment
    for segment in segments:
        # Tokenize the segment
        input_tokens = tokenizer(segment, return_tensors="pt")

        # Generate the translation
        translated_tokens = model.generate(**input_tokens)

        # Decode the translated tokens
        segment_translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

        # Append segment translation to the overall translation
        translated_text += segment_translation

    return translated_text

# if __name__ == "__main__":
#     file_path = "/path/to/input/file.txt"
#     translated_text = translate_file(file_path)
#     print(f"Translated text:\n{translated_text}")

if __name__ == "__main__":
    file_path = "/content/bonalufestival(1).txt"
    translated_text = translate_file(file_path)
    print(f"Translated text:\n{translated_text}")


Translated text:
बाल देवता की उपासना करने के लिए धार्मिक त्योहार, भारत के दक्षिण राज्य में मनाया जाता है। इस पर्व को माता देवी देवी, महलाली के लिए समर्पित किया जाता है, जो लोगों को दुष्ट शक्‍तियों से सुरक्षित रखने के लिए माना जाता है। ब्बला के महीने में, जो जुलाई और अगस्त के बीच आता है ।44 शहर, जहां यह Hakakia राजवंश द्वारा मनाया गया था. माना जाता है कि पर्ब्ब 12वीं सदी में कातर्टिया वंश के शासन के दौरान शुरू हो गया है। कथा के अनुसार, देवी महल महल महला ने राजा के लिए एक सपना देखा और उससे कहा कि लोगों को एक घातक महामारी से बचाने के लिए Baphroual के राजा की रक्षा करने के लिए।जो स्त्रियाँ मिट्टी के बर्तन ले जा रही हैं, चावल, दालेंदार, और पानी से भरे हुए, नीम के पत्तों से सजाया जाता है, और पानी से सजाया जाता है. बर्तन सिर पर ले जाते हैं, और स्त्रियों को ढोल बजाने, देवी के लिए बलि बनाने के लिए नृत्य किया जाता है. बाद में इन्हें लोगों के बीच एक महत्त्वपूर्ण त्योहार के रूप में वितरित किया जाता है.आम तौर पर लोगों को प्रचार करने में हिस्सा लेने में बड़ी भूमिका निभाई है । इस पर्व ने तेलानाना की 

In [4]:
from googletrans import Translator

def translate_file(file_path, src_lang="en", tgt_lang="hi"):
    translator = Translator()

    # Read file contents
    with open(file_path, 'r') as f:
        text = f.read()

    # Translate the text
    translated_text = translator.translate(text, src=src_lang, dest=tgt_lang).text

    return translated_text

if __name__ == "__main__":
    file_path = "/content/bonalufestival(1).txt"
    translated_text = translate_file(file_path)
    print(f"Translated text:\n{translated_text}")


Translated text:
बोनालु एक महत्वपूर्ण हिंदू त्योहार है जो दक्षिणी राज्य तेलंगाना, भारत में मनाया जाता है।यह त्योहार माँ देवी, महाकली की पूजा के लिए समर्पित है, जो लोगों को बुरी ताकतों से बचाने के लिए माना जाता है।बोनालु को अशादा के महीने में मनाया जाता है, जो जुलाई और अगस्त के बीच आता है।त्योहार सांस्कृतिक विरासत और धार्मिक सद्भाव का प्रतीक है और जीवन के सभी क्षेत्रों के लोगों द्वारा बड़े उत्साह के साथ मनाया जाता है।

बोनालु के त्योहार की उत्पत्ति प्राचीन शहर हैदराबाद में है, जहां इसे चालुक्य राजवंश द्वारा मनाया गया था।माना जाता है कि यह त्योहार 12 वीं शताब्दी में काकती राजवंश के शासनकाल के दौरान शुरू हुआ था।किंवदंती के अनुसार, देवी महाकाली राजा के एक सपने में दिखाई दी और उन्हें लोगों को एक घातक महामारी से बचाने के लिए बोनालु उत्सव करने के लिए कहा।राजा ने देवी के निर्देशों का पालन किया और त्योहार का प्रदर्शन किया, जो अभी भी तेलंगाना में बड़े उत्साह के साथ मनाया जाता है।

बोनालु उत्सव उन महिलाओं द्वारा मनाया जाता है जो चावल, गुड़, दही और पानी से भरे मिट्टी के बर्तन ले जाते हैं, नीम के प

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from googletrans import Translator

def translate_file(file_path, src_lang="en", tgt_lang="hi"):
    translator = Translator()

    # Read file contents
    with open(file_path, 'r') as f:
        text = f.read()

    try:
        # Translate the text
        translated_text = translator.translate(text, src=src_lang, dest=tgt_lang).text
    except Exception as e:
        print("Translation error:", str(e))
        return None

    return translated_text

def save_to_file(file_path, content):
    with open(file_path, 'w') as f:
        f.write(content)

if __name__ == "__main__":
    input_file_path = "/content/bonalufestival(1).txt"
    output_file_path = "/content/translated_file.txt"
    
    translated_text = translate_file(input_file_path)
    
    if translated_text is not None:
        save_to_file(output_file_path, translated_text)
        print(f"Translated text saved to: {output_file_path}")
        print(f"Translated text:\n{translated_text}")
    else:
        print("Translation failed. Please check the error message.")


Translated text saved to: /content/translated_file.txt
Translated text:
बोनालु एक महत्वपूर्ण हिंदू त्योहार है जो दक्षिणी राज्य तेलंगाना, भारत में मनाया जाता है।यह त्योहार माँ देवी, महाकली की पूजा के लिए समर्पित है, जो लोगों को बुरी ताकतों से बचाने के लिए माना जाता है।बोनालु को अशादा के महीने में मनाया जाता है, जो जुलाई और अगस्त के बीच आता है।त्योहार सांस्कृतिक विरासत और धार्मिक सद्भाव का प्रतीक है और जीवन के सभी क्षेत्रों के लोगों द्वारा बड़े उत्साह के साथ मनाया जाता है।

बोनालु के त्योहार की उत्पत्ति प्राचीन शहर हैदराबाद में है, जहां इसे चालुक्य राजवंश द्वारा मनाया गया था।माना जाता है कि यह त्योहार 12 वीं शताब्दी में काकती राजवंश के शासनकाल के दौरान शुरू हुआ था।किंवदंती के अनुसार, देवी महाकाली राजा के एक सपने में दिखाई दी और उन्हें लोगों को एक घातक महामारी से बचाने के लिए बोनालु उत्सव करने के लिए कहा।राजा ने देवी के निर्देशों का पालन किया और त्योहार का प्रदर्शन किया, जो अभी भी तेलंगाना में बड़े उत्साह के साथ मनाया जाता है।

बोनालु उत्सव उन महिलाओं द्वारा मनाया जाता है जो चावल, गुड़, द

In [16]:
from googletrans import Translator
import textwrap

def translate_file(file_path, src_lang="en", tgt_lang="hi"):
    translator = Translator()

    # Read file contents
    with open(file_path, 'r') as f:
        text = f.read()

    # Split the text into smaller chunks
    max_chunk_size = 4000
    text_chunks = textwrap.wrap(text, max_chunk_size)

    translated_text = ""
    try:
        # Translate the text chunks
        for chunk in text_chunks:
            translated_chunk = translator.translate(chunk, src=src_lang, dest=tgt_lang).text
            translated_text += translated_chunk + "\n"  # Add line break at the end of each translated chunk

    except Exception as e:
        print("Translation error:", str(e))
        return None

    return translated_text


def save_to_file(file_path, content):
    with open(file_path, 'w') as f:
        f.write(content)


if __name__ == "__main__":
    input_file_path = "/content/bonalufestival.txt"
    output_file_path = "/content/translated_file.txt"

    translated_text = translate_file(input_file_path)

    if translated_text is not None:
        save_to_file(output_file_path, translated_text)
        print(f"Translated text saved to: {output_file_path}")
        print(f"Translated text:\n{translated_text}")
    else:
        print("Translation failed. Please check the error message", str(e))


Translated text saved to: /content/translated_file.txt
Translated text:
बोनालु एक महत्वपूर्ण हिंदू त्योहार है जो दक्षिणी राज्य तेलंगाना, भारत में मनाया जाता है।यह त्योहार माँ देवी, महाकली की पूजा के लिए समर्पित है, जो लोगों को बुरी ताकतों से बचाने के लिए माना जाता है।बोनालु को अशादा के महीने में मनाया जाता है, जो जुलाई और अगस्त के बीच आता है।त्योहार सांस्कृतिक विरासत और धार्मिक सद्भाव का प्रतीक है और जीवन के सभी क्षेत्रों के लोगों द्वारा बड़े उत्साह के साथ मनाया जाता है।बोनालु के त्योहार की उत्पत्ति प्राचीन शहर हैदराबाद में है, जहां इसे चालुक्य राजवंश द्वारा मनाया गया था।माना जाता है कि यह त्योहार 12 वीं शताब्दी में काकती राजवंश के शासनकाल के दौरान शुरू हुआ था।किंवदंती के अनुसार, देवी महाकाली राजा के एक सपने में दिखाई दी और उन्हें लोगों को एक घातक महामारी से बचाने के लिए बोनालु उत्सव करने के लिए कहा।राजा ने देवी के निर्देशों का पालन किया और त्योहार का प्रदर्शन किया, जो अभी भी तेलंगाना में बड़े उत्साह के साथ मनाया जाता है।बोनालु उत्सव उन महिलाओं द्वारा मनाया जाता है जो चावल, गुड़, दही औ

In [15]:
from googletrans import Translator

def translate_file(file_path, src_lang="en", tgt_lang="hi"):
    translator = Translator()

    # Read file contents
    with open(file_path, 'r') as f:
        lines = f.readlines()

    translated_lines = []
    try:
        # Translate the lines
        for line in lines:
            translated_line = translator.translate(line, src=src_lang, dest=tgt_lang).text
            translated_lines.append(translated_line)

    except Exception as e:
        print("Translation error:", str(e))
        return None

    # Join the translated lines
    translated_text = "\n".join(translated_lines)
    return translated_text

def save_to_file(file_path, content):
    with open(file_path, 'w') as f:
        f.write(content)

if __name__ == "__main__":
    input_file_path = "/content/bonalufestival.txt"
    output_file_path = "/content/translated_file.txt"
    
    translated_text = translate_file(input_file_path)
    
    if translated_text is not None:
        save_to_file(output_file_path, translated_text)
        print(f"Translated text saved to: {output_file_path}")
        print(f"Translated text:\n{translated_text}")
    else:
        print("Translation failed. Please check the error message.")


Translation error: list index out of range
Translation failed. Please check the error message.
